In [30]:
import requests

def team_info(team_id: int, season: int, sport: str):
    if sport not in ('xc','tfo','tfi'): return 'error'
    td_sport = 'tf' if sport == 'tfo' else sport
    team_data = requests.get('https://www.athletic.net/api/v1/TeamNav/Team',params=dict(team=team_id,sport=td_sport,season=season))

    team_core = requests.get('https://www.athletic.net/api/v1/TeamHome/GetTeamCore',dict(teamId=team_id,sport=sport,season=season))
    anettokens = team_core.json()['jwtTeamHome']
    referer_url = f'https://www.athletic.net/team/{team_id}/cross-country/{season}'
    headers = {'referer':referer_url,'anettokens':anettokens}
    roster = requests.get('https://www.athletic.net/api/v1/TeamHome/GetAthletes',headers = headers, params = dict(seasonID = season))
    team_info = team_data.json()['team']

    schedule = requests.get('https://www.athletic.net/api/v1/TeamHomeCal/GetCalendar',headers = headers, params = dict(seasonID = season))
    schedule_output = [
        {
            'meet_name':meet['Name'],
            'anet_meet_id': meet['MeetID'], 
            'street_address':meet['StreetAddress'],
            'city': meet['City'], 
            'state': meet['State'],
            'zipcode': meet['PostalCode'],
            'location': meet['Location'],
            'date': meet['Date']
            
        } 
        for meet in schedule.json()
    ]
    return {
                'name' : team_info['Name'],
                'city' : team_info['City'],
                'state' : team_info['State'],
                'mascot' : team_info['Mascot'],
                'roster' : roster.json(),
                'schedule' : schedule_output#.json()
            }

team_info(493,2023,'xc')

{'name': 'Garfield',
 'city': 'Seattle',
 'state': 'WA',
 'mascot': 'Bulldogs',
 'roster': [{'ID': 17657892, 'Name': 'Aaron Johnson', 'Gender': 'M'},
  {'ID': 21370434, 'Name': 'Adam Shukis', 'Gender': 'M'},
  {'ID': 21370415, 'Name': 'Aislin Finley', 'Gender': 'F'},
  {'ID': 23753866, 'Name': 'Alexander Miller', 'Gender': 'M'},
  {'ID': 21370417, 'Name': 'Amelia Hawkins', 'Gender': 'F'},
  {'ID': 17657871, 'Name': 'Amelie Igel', 'Gender': 'F'},
  {'ID': 23753846, 'Name': 'Amina Buribayeva', 'Gender': 'F'},
  {'ID': 23753871, 'Name': 'Angelina Shapiro', 'Gender': 'F'},
  {'ID': 23753856, 'Name': 'Anneka Herring', 'Gender': 'F'},
  {'ID': 17657868, 'Name': 'Anyanka Ceballos-Baliga', 'Gender': 'F'},
  {'ID': 23753862, 'Name': 'Ardon Liffmann', 'Gender': 'M'},
  {'ID': 22211896, 'Name': 'Ava Stewart', 'Gender': 'F'},
  {'ID': 23753851, 'Name': 'Benjamin Frisse', 'Gender': 'M'},
  {'ID': 22211747, 'Name': 'Benny Carlson', 'Gender': 'F'},
  {'ID': 19608500, 'Name': 'Benny Smith-Roberts', 'G

In [79]:
def get_meet_results(meet_id: int, sport: str):
    params = dict(meetId=meet_id, sport=sport)
    meet = requests.get('https://www.athletic.net/api/v1/Meet/GetMeetData',params=params)
    
    meet_payload = meet.json()
    
    meet_data = {
        'meet' :
            {
                'location' : meet_payload['meet']['Location']['Name'],
                'street_address' : meet_payload['meet']['Location']['Address'],
                'city' : meet_payload['meet']['Location']['City'],
                'state' : meet_payload['meet']['Location']['State'],
                'zipcode' : meet_payload['meet']['Location']['PostalCode']
            },
        'races': [
                    {
                        'race_id': race['IDMeetDiv'],
                        'race' : race['DivName'],
                        'division' : race['Division'],
                        'place_depth' : race['PlaceDepth'],
                        'score_depth' : race['ScoreDepth'],
                        'start_time' : race['RaceTime'],
                        'results' : [],
                        'team_scores' : []
                    }   
                    for race in meet_payload['xcDivisions']
        ]
    }
    
    results = requests.get('https://www.athletic.net/api/v1/Meet/GetAllResultsData', headers = dict(referer = 'https://www.athletic.net/CrossCountry/meet/225886/results/all',anettokens = meet_payload['jwtMeet']))
    
    for finisher in results.json()['results']:
        finisher_details = {
            'athlete_id' : finisher['AthleteID'],
            'name' : finisher['FirstName'] + ' ' + finisher['LastName'],
            'team_id' : finisher['TeamID'],
            'team' : finisher['SchoolName'],
            'grade' : finisher['AgeGrade'],
            'result' : finisher['Result'],
            'place' : finisher['Place']
        }
        race_id = finisher['RaceDivisionID']
        for race in meet_data['races']:
            if race_id == race['race_id']:
                race['results'].append(finisher_details)
                
    for team_score in results.json()['teamScores']:
        
        
    return meet_data
        

In [80]:
get_meet_results(meet_id = 225886, sport = 'xc')

{'meet': {'location': 'Lower Woodland',
  'street_address': 'Green Lake Way N & N 50th St, Seattle, WA  ',
  'city': 'Seattle',
  'state': 'WA',
  'zipcode': '98103'},
 'races': [{'race_id': 902289,
   'race': '5,000 Meters Girls Open',
   'division': 'Girls Open',
   'place_depth': 7,
   'score_depth': 5,
   'start_time': '2023-09-15T15:40:00',
   'results': [{'athlete_id': 18902065,
     'name': 'Zoe Louis',
     'team_id': 415,
     'team': 'Bishop Blanchet',
     'grade': '11',
     'result': '18:59.0',
     'place': 1},
    {'athlete_id': 18902058,
     'name': 'Lily LaPorte',
     'team_id': 415,
     'team': 'Bishop Blanchet',
     'grade': '11',
     'result': '19:10.3',
     'place': 2},
    {'athlete_id': 18929410,
     'name': 'Sophia Bazzi',
     'team_id': 419,
     'team': 'Holy Names Academy',
     'grade': '11',
     'result': '20:02.2',
     'place': 3},
    {'athlete_id': 23662046,
     'name': 'Chloe Cromwell',
     'team_id': 415,
     'team': 'Bishop Blanchet',
   

In [46]:
race_results

{902289: [{'IDResult': 48119313,
   'AthleteName': None,
   'TeamName': None,
   'Result': '18:59.0',
   'Place': 1,
   'Comment': None,
   'Official': 1,
   'SortValue': 1139.0,
   'AgeGrade': '11',
   'AthleteID': 18902065,
   'FirstName': 'Zoe',
   'LastName': 'Louis',
   'Gender': 'F',
   'TeamID': 415,
   'RaceDivisionID': 902289,
   'sr': False,
   'pr': False,
   'Grade': '11',
   'Row': 1,
   'EditorID': -385145,
   'SchoolName': 'Bishop Blanchet',
   'MediaCount': {},
   'ShortCode': 'yoFPQ6oF2NJqY'},
  {'IDResult': 48119314,
   'AthleteName': None,
   'TeamName': None,
   'Result': '19:10.3',
   'Place': 2,
   'Comment': None,
   'Official': 1,
   'SortValue': 1150.3,
   'AgeGrade': '11',
   'AthleteID': 18902058,
   'FirstName': 'Lily',
   'LastName': 'LaPorte',
   'Gender': 'F',
   'TeamID': 415,
   'RaceDivisionID': 902289,
   'sr': False,
   'pr': False,
   'Grade': '11',
   'Row': 2,
   'EditorID': -385145,
   'SchoolName': 'Bishop Blanchet',
   'MediaCount': {},
   'Sho

In [ ]:
r2.json()

In [ ]:
r4 = requests.get('https://www.athletic.net/api/v1/SignedInUser/GetSignedInUser',headers={'referer':'https://www.athletic.net/team/123/cross-country/2023'})

In [ ]:
r4.json()

In [ ]:
from typing import Optional, List

from sqlmodel import Field, Relationship, Session, SQLModel, create_engine

from datetime import date, time

In [ ]:
class Athlete(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    anet_id: int
    name: str
    gender: str

    results: List['Result'] = Relationship(back_populates = 'athlete')

class Team(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    anet_id: int
    name: str

class Meet(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    anet_id: int
    name: str
    location: str
    date: date

class Result(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    anet_id: int
    result: time
    place: int

    athlete_id: Optional[int] = Field(default=None, foreign_key = 'athlete.id')
    athlete: Optional[Athlete] = Relationship(back_populates = 'results')
    
    team_id: Optional[int] = Field(default=None, foreign_key = 'team.id')
    meet_id: Optional[int] = Field(default=None, foreign_key = 'meet.id')


db_name = "anet_results"
db_string = f"postgresql:///{db_name}"

engine = create_engine(db_string, echo=True)


def create_db_and_tables():
    SQLModel.metadata.create_all(engine)

In [ ]:
create_db_and_tables()

In [ ]:
session = Session(engine)

for result in r2.json():
    athlete = Athlete(anet_id = result['ID'],name = result['Name'], gender = result['Gender'])
    session.add(athlete)

session.commit()

In [ ]:
session.close()